# Maestria en Explotacion de datos y Descubrimiento de conocimiento
### Sistemas de información geografica
### Trabajo Practico N°2

# Clasificación por pixel usando deep learning

In [16]:
%load_ext autoreload
%autoreload 2

In [17]:
import sys
sys.path.append('../lib')

In [18]:
import pandas as pd

## Constantes globales

In [19]:
RESULT_PATH  = "../results"

## Funciones helper

In [20]:
result_path = lambda file: f'{RESULT_PATH}/{file}'

In [21]:
df = pd.read_csv(result_path('dataset.csv'))

In [22]:
df = df.loc[:, ~df.columns.isin(['GEOMETRY', 'Unnamed: 0', 'in1', 'ogc_fid', 'originfid'])]
df

,id,cultivo,band_0,band_1,band_2,band_3,band_4,band_5,band_6,band_7,...,band_56,band_57,band_58,band_59,band_60,band_61,band_62,band_63,band_64,band_65
0,2,MAIZ,0.326524,0.909134,1.969667,0.233421,-1.020433,0.206807,0.213882,0.663262,...,0.891177,1.833652,0.100895,-2.466148,0.076450,0.078313,0.647099,0.397356,0.239041,0.231089
1,20,CAMPONATUR,0.346409,0.920005,2.060019,0.207121,-1.373308,0.174691,0.181553,0.673205,...,0.945336,2.298482,0.166044,-2.057782,0.130389,0.135423,0.696830,0.654610,0.608975,0.463962
2,3,MAIZ,0.257845,0.870543,1.694857,0.201694,-0.871542,0.183388,0.187610,0.628923,...,0.896695,1.873821,0.149455,-1.777502,0.120290,0.124158,0.652031,0.420873,0.429346,0.250323
3,5,GIRASOL,0.319363,0.905186,1.938422,0.229047,-1.016873,0.203057,0.209811,0.659681,...,0.941656,2.261128,0.104525,-2.876130,0.077190,0.079203,0.693358,0.635038,0.330645,0.444570
4,1,SOJA,0.354500,0.924392,2.098372,0.248241,-1.050501,0.218901,0.227279,0.677250,...,0.900739,1.904153,0.164812,-1.628484,0.134796,0.139384,0.655666,0.438454,0.510339,0.264981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,2,MAIZ,0.308442,0.899134,1.892022,0.211353,-1.112663,0.184684,0.190823,0.654221,...,0.913431,2.004608,0.103744,-2.641459,0.077703,0.079685,0.667178,0.495641,0.289326,0.314287
462,2,MAIZ,0.332909,0.912638,1.998092,0.192704,-1.443068,0.161102,0.167138,0.666454,...,0.967863,2.550877,0.110279,-2.986429,0.080992,0.083225,0.718379,0.782548,0.380867,0.596934
463,2,MAIZ,0.424754,0.961641,2.476773,0.141078,-2.506952,0.106937,0.110625,0.712377,...,1.005820,3.095617,0.130889,-2.958926,0.096496,0.099644,0.755837,1.037006,0.503061,0.889347
464,3,MAIZ,0.370429,0.932968,2.176768,0.260183,-1.037813,0.229863,0.239096,0.685215,...,0.967523,2.546729,0.204686,-1.836522,0.164312,0.171939,0.718050,0.780502,0.854688,0.594726
